In [ ]:
pip install opencv-python

In [16]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
from skimage import io,measure
from scipy import ndimage as nd
from skimage.color import label2rgb

In [ ]:
i=input("enter the image path:")
img = io.imread(i)
plt.imshow(img)

# A required callback method that goes into the trackbar function.
def nothing(x):
    pass

# Load the image from a file
#image_path = "your_image_file.jpg"  # Specify the path to your image
frame = cv2.imread(i)

if frame is None:
    print("Error loading image")
    exit()

# Create a window named trackbars.
cv2.namedWindow("Trackbars")

# Create 6 trackbars that will control the lower and upper range of H, S and V channels.
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)

while True:
    # Convert the BGR image to HSV image.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get the new values of the trackbar in real time as the user changes them
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")

    # Set the lower and upper HSV range according to the value selected by the trackbar
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    print(lower_range)
    print(upper_range)

    # Create a mask using the specified HSV range
    mask = cv2.inRange(hsv, lower_range, upper_range)

    # Visualize the real part of the target color (Optional)
    res = cv2.bitwise_and(frame, frame, mask=mask)

    # Converting the binary mask to a 3-channel image
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # Stack the mask, original frame, and the filtered result
    stacked = np.hstack((mask_3, frame, res))

    # Show this stacked frame at 40% of the size.
    cv2.imshow('Trackbars', cv2.resize(stacked, None, fx=0.4, fy=0.4))

    # If the user presses ESC then exit the program
    key = cv2.waitKey(1)
    if key == 27:  # ESC key
        break

# Display the final mask using matplotlib
plt.imshow(mask, cmap='gray')
plt.title('Binary Mask')
#plt.axis('off')  # Hide axis
plt.show()

# Destroy the windows.
cv2.destroyAllWindows()


In [ ]:
closed_mask=nd.binary_closing(mask,np.ones((7,7)))
plt.imshow(closed_mask)


In [ ]:
label=measure.label(closed_mask)
plt.imshow(label)

In [ ]:
img_label=label2rgb(label,image=img)
plt.imshow(img_label)

In [ ]:
from skimage import measure
import pandas as pd

props = measure.regionprops_table(label, img,
                                   properties=['label', 'area', 'equivalent_diameter', 'mean_intensity', 'solidity'])
df = pd.DataFrame(props)
print(df.head())
